# ERA V4 – Session 9: Sanity Test (Imagenette)
This Colab notebook verifies the training pipeline for ResNet models using timm.
- Dataset: Imagenette 320px
- Model: ResNet18 (1 epoch)
- Environment: CUDA GPU (Colab)


In [1]:
!pip -q install timm==1.* wandb==0.* pandas matplotlib
!wget -q https://s3.amazonaws.com/fast-ai-imageclas/imagenette2-320.tgz
!tar -xzf imagenette2-320.tgz
!ls /content/imagenette2-320/train | head


n01440764
n02102040
n02979186
n03000684
n03028079
n03394916
n03417042
n03425413
n03445777
n03888257


In [2]:
!mkdir -p /content/runs
!rm -rf timm_src
!git clone -q https://github.com/huggingface/pytorch-image-models.git timm_src
%cd timm_src

!python train.py \
  --data-dir /content/imagenette2-320 \
  --train-split train --val-split val \
  --model resnet18 --epochs 1 --batch-size 64 \
  --opt sgd --momentum 0.9 --weight-decay 1e-4 \
  --sched cosine --warmup-epochs 0 --lr 0.01 \
  --amp --channels-last --pin-mem --workers 2 \
  --output /content/runs --experiment sanity_run \
  --checkpoint-hist 1 --save-images --log-interval 10 \
  | tee /content/runs/sanity_log.txt


/content/timm_src
Training with a single process on 1 device (cuda).
Model resnet18 created, param count:11689512
Data processing configuration for current model + dataset:
	input_size: (3, 224, 224)
	interpolation: bicubic
	mean: (0.485, 0.456, 0.406)
	std: (0.229, 0.224, 0.225)
	crop_pct: 0.95
	crop_mode: center
Created SGD (sgd) optimizer: lr: 0.01, momentum: 0.9, dampening: 0, weight_decay: 0.0001, nesterov: True, maximize: False, foreach: None, differentiable: False, fused: None
Using native Torch AMP. Training in mixed precision.
Scheduled epochs: 1 (epochs + cooldown_epochs). Warmup within epochs when warmup_prefix=False. LR stepped per epoch.
Train: 0 [   0/147 (  1%)]  Loss: 7.02 (7.02)  Time: 7.777s,    8.23/s  (7.777s,    8.23/s)  LR: 1.000e-02  Data: 0.838 (0.838)
Train: 0 [  10/147 (  7%)]  Loss: 4.28 (5.81)  Time: 0.153s,  418.46/s  (0.884s,   72.36/s)  LR: 1.000e-02  Data: 0.096 (0.181)
Train: 0 [  20/147 ( 14%)]  Loss: 2.93 (4.62)  Time: 0.211s,  303.86/s  (0.591s,  108

In [3]:
import glob, os
print("✅ Checkpoints:", glob.glob("/content/runs/sanity_run/*.pth.tar"))
!tail -n 20 /content/runs/sanity_log.txt


✅ Checkpoints: ['/content/runs/sanity_run/checkpoint-0.pth.tar', '/content/runs/sanity_run/model_best.pth.tar', '/content/runs/sanity_run/last.pth.tar']
--result
[
    {
        "epoch": 0,
        "train": {
            "loss": 3.0787709991948136
        },
        "validation": {
            "loss": 1.977895087709852,
            "top1": 30.063694263628335,
            "top5": 81.83439492584034
        }
    }
]
